In [1]:
#import dependencies
import pandas as pd
import json
from flask import Flask, render_template,jsonify

# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float ,Date

from sqlalchemy.orm import Session

from sqlalchemy import inspect
from sqlalchemy.ext.automap import automap_base

from sqlalchemy import create_engine, inspect, func, desc, extract, select, Table
from collections import defaultdict



In [3]:

# ===========================Database Connection========================

# Create Database Connection
engine = create_engine("sqlite:///censusdata.sqlite", echo=False)
Base = automap_base()
Base.prepare(engine, reflect=True)
session = Session(engine)

conn = engine.connect()

In [4]:
query = "select CountyID,CountyName,TotalPopulation,Year, Over149999 as Over150000, Over200000, 'AIAN' as Race from census_aian union "
query += "select CountyID,CountyName,TotalPopulation,Year, Over150000, Over200000, 'Asian' as Race from census_asian where race = 'Asian' union "
query += "select CountyID,CountyName,TotalPopulation,Year, Over150000, Over200000, 'Black' as Race from census_black union "
query += "select CountyID,CountyName,TotalPopulation,Year, Over150000, Over200000, 'Hispanic' as Race from census_hispanic union "
query += "select CountyID,CountyName,TotalPopulation,Year, Over150000, Over200000, 'Mixed' as Race from census_mixed union "            
query += "select CountyID,CountyName,TotalPopulation,Year, Over150000, Over200000, 'Other' as Race from census_other union "                           
query += "select CountyID,CountyName,TotalPopulation,Year, Over150000, Over200000, 'Whites' as Race from census_whites order by CountyName, Year"                           
df_all = pd.read_sql_query(query, conn)
# df_all[df_all.CountyName=='Los Angeles County, California']
df_all.head()

,CountyID,CountyName,TotalPopulation,Year,Over150000,Over200000,Race
0,45001,"Abbeville County, South Carolina",8,2000,0,0,Asian
1,45001,"Abbeville County, South Carolina",27,2000,0,0,Other
2,45001,"Abbeville County, South Carolina",55,2000,0,0,Hispanic
3,45001,"Abbeville County, South Carolina",59,2000,0,0,Mixed
4,45001,"Abbeville County, South Carolina",2903,2000,0,5,Black


In [5]:
df_all['CountyPopulation'] = df_all['TotalPopulation'].groupby([df_all['CountyName'],df_all['Year']]).transform('sum')
df_all['TotalPercentage'] = (df_all["TotalPopulation"]/df_all['CountyPopulation'])*100
df_all.head(15)

,CountyID,CountyName,TotalPopulation,Year,Over150000,Over200000,Race,CountyPopulation
0,45001,"Abbeville County, South Carolina",8,2000,0,0,Asian,10188
1,45001,"Abbeville County, South Carolina",27,2000,0,0,Other,10188
2,45001,"Abbeville County, South Carolina",55,2000,0,0,Hispanic,10188
3,45001,"Abbeville County, South Carolina",59,2000,0,0,Mixed,10188
4,45001,"Abbeville County, South Carolina",2903,2000,0,5,Black,10188
5,45001,"Abbeville County, South Carolina",7136,2000,52,22,Whites,10188
6,22001,"Acadia Parish, Louisiana",12,2000,0,0,Other,21228
7,22001,"Acadia Parish, Louisiana",53,2000,0,0,Asian,21228
8,22001,"Acadia Parish, Louisiana",66,2000,0,0,Mixed,21228
9,22001,"Acadia Parish, Louisiana",92,2000,0,12,Hispanic,21228


In [12]:
df_all['Over150Population'] = df_all['Over150000'].groupby([df_all['CountyName'],df_all['Year']]).transform('sum')
df_all['Over150Percentage'] = (df_all["Over150000"]/df_all['Over150Population'])*100
df_all['Over200Population'] = df_all['Over200000'].groupby([df_all['CountyName'],df_all['Year']]).transform('sum')
df_all['Over200Percentage'] = (df_all["Over200000"]/df_all['Over200Population'])*100
# df_all.tail(15)
df_all[df_all.CountyName=='Los Angeles County, California']

,CountyID,CountyName,TotalPopulation,Year,Over150000,Over200000,Race,CountyPopulation,TotalPercentage,Over150Percentage,Over150Population,Over200Population,Over200Percentage
14880,6037,"Los Angeles County, California",14517,2000,766,212,AIAN,4136467,0.350952,0.779311,98292,119418,0.177528
14881,6037,"Los Angeles County, California",137746,2000,2441,2802,Mixed,4136467,3.330040,2.483417,98292,119418,2.346380
14882,6037,"Los Angeles County, California",345153,2000,4697,4628,Black,4136467,8.344150,4.778619,98292,119418,3.875463
14883,6037,"Los Angeles County, California",361944,2000,12022,10144,Asian,4136467,8.750076,12.230904,98292,119418,8.494532
14884,6037,"Los Angeles County, California",511505,2000,3789,3864,Other,4136467,12.365746,3.854841,98292,119418,3.235693
14885,6037,"Los Angeles County, California",1012555,2000,10023,10697,Hispanic,4136467,24.478740,10.197168,98292,119418,8.957611
14886,6037,"Los Angeles County, California",1753047,2000,64554,87071,Whites,4136467,42.380297,65.675742,98292,119418,72.912794
14887,6037,"Los Angeles County, California",16970,2005,566,903,AIAN,4311069,0.393638,0.377049,150113,161960,0.557545
14888,6037,"Los Angeles County, California",57515,2005,2145,2459,Mixed,4311069,1.334124,1.428924,150113,161960,1.518276
14889,6037,"Los Angeles County, California",331807,2005,7870,6333,Black,4311069,7.696629,5.242717,150113,161960,3.910225
